In [ ]:
!pip install roboflow
import os
from roboflow import Roboflow
rf = Roboflow(api_key="MKjiwHcUxN5wVAcYZXrE")
project = rf.workspace("mohammad-amin-asadi-1tacf").project("bone-fracture-vqdiz")
version = project.version(2)
dataset = version.download("yolo11")


In [ ]:
# prompt: give me code for data augmentation for the above dataset

import os
import random
import cv2
import numpy as np

def augment_data(image_path, output_path, num_augmentations=5):
    """
    Performs data augmentation on a given image.

    Args:
        image_path: Path to the input image.
        output_path: Directory to save the augmented images.
        num_augmentations: Number of augmented images to generate.
    """

    img = cv2.imread(image_path)

    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    image_name, _ = os.path.splitext(os.path.basename(image_path))
    for i in range(num_augmentations):
        augmented_image = img.copy()
        # Random Rotation
        rows, cols, _ = augmented_image.shape
        rotation_angle = random.randint(-15, 15)  # Rotate by up to 15 degrees
        rotation_matrix = cv2.getRotationMatrix2D((cols/2, rows/2), rotation_angle, 1)
        augmented_image = cv2.warpAffine(augmented_image, rotation_matrix, (cols, rows))

        # Random Horizontal Flip
        if random.random() < 0.5:
            augmented_image = cv2.flip(augmented_image, 1)  # Flip horizontally

        # Random Brightness Adjustment
        brightness_factor = random.uniform(0.8, 1.2) # Adjust brightness slightly
        augmented_image = cv2.convertScaleAbs(augmented_image, alpha=brightness_factor, beta=0)

        #Save the augmented image
        output_file_path = os.path.join(output_path, f"{image_name}_augmented_{i+1}.jpg")
        cv2.imwrite(output_file_path, augmented_image)

# Example Usage (assuming your images are in 'train/images')
image_dir = os.path.join(dataset.location, "train", "images")
output_dir = os.path.join(dataset.location, "augmented_images")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')): # add more extensions if needed
        image_path = os.path.join(image_dir, filename)
        augment_data(image_path, output_dir, num_augmentations=2) # Generate 2 augmented images per original image




In [ ]:

# Step 3: Install the required YOLOv11 library
!pip install ultralytics

# Step 4: Import the YOLO class
from ultralytics import YOLO

# Step 5: Load and train the YOLOv11 model
# Load a pre-trained YOLOv11 model. Here we use YOLOv8n (nano version).
# model = YOLO("yolo11n.pt")
model = YOLO("yolo11s.pt")

In [ ]:
import os
dataset_dir = "/content/bone-fracture-2"
data_yaml_path = os.path.join(dataset_dir, "data.yaml")

# Step 2: Initialize YOLOv11 model
model = YOLO("best2130e.pt")  # 'yolov11.yaml' is specified for bone fracture detection

# Step 3: Train the model
# Ensure the dataset YAML file contains the correct paths and configuration

model.train(
    data=data_yaml_path,  # Path to the dataset configuration
    epochs=100,           # Number of epochs to train
    imgsz=640,           # Image size for training
    batch=16,            # Batch size
    name="bone_fracture_model5"  # Name of the training run
)

# Step 4: Evaluate the model
results = model.val()

# Step 5: Save the trained model
onnx_export_path = model.export(format="onnx")  # Export model to ONNX format for deployment
pt_export_path = "runs/train/bone_fracture_model/weights/best.pt"  # Path to the saved .pt model


In [ ]:

# !pip install ultralytics
# from ultralytics import YOLO
# import os

# Step 1: Load the trained YOLOv8 model
model = YOLO('best2155e.pt')  # Replace with your model's path

# Step 2: Path to the image you want to run inference on
image_path = "fracture8.jpeg"  # Replace with your actual image path

# Step 3: Run inference on the image
results = model.predict(source=image_path)  # Run inference

# Step 4: Check for detections
if len(results[0].boxes) > 0:  # If any boxes are detected
    results[0].show()  # Show the image with bounding boxes
else:
    print("No detections found.")

# Step 5: Save the results (predicted image with bounding boxes)
results[0].save()  # This will save the annotated image with predictions to the default save directory

# Optional: Print out the predictions
for pred in results[0].boxes.data:  # Loop through each prediction
    class_id = int(pred[5])  # The predicted class ID
    confidence = pred[4].item()  # The confidence score
    bbox = pred[:4].cpu().numpy()  # The bounding box coordinates (x, y, width, height)
    print(f"Class ID: {class_id}, Confidence: {confidence:.4f}, Bounding Box: {bbox}")